In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### define coupled variables to integrate ! :D


In [ ]:
def dydx(x,y):
    
    #y is an array containing all the variables 
    #being differentiated ! :D
    
    # our equation is d^2y/dx^2 = -y ! :D
    
    #dy/dx = z
    #dz/dx = -y
    
    #let y = y[0]
    #let z = y[1]
    
    #declare an array that has derivatives of our variables ! :D
    
    y_derivs = np.zeros(2)
    
    #dy/dx = z
    
    y_derivs[0] = y[1]
    
    #dz/dx = -y
    
    y_derivs[1] = -1*y[0]
    
    #this functions returns an array containing the derivatives
    #of each variable! :D
    return y_derivs



### time for runga kutta core! 

In [ ]:
def rk5_mv_core(dydx, xi, yi, nv, h):
    
    #xi = current value of x
    #yi = current value of each variable
    #nv = number of variable
    #h = step size
    #declare the k arrays. each contains k value for every variable
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define xc2
    x_c2 = xi + 0.2*h 
    
    #define xc3
    x_c3 = xi + 0.3*h
    
    #define xc4
    x_c4 = xi + 0.6*h
    
    #define xc5
    x_c5 = xi + h
    
    #define xc6 
    x_c6 = xi + 0.875*h
    
    #define x at 1/2 way point
    #x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    #x_ipo = xi+ h
    
    #declare a temp y array. contains values of y at different places
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.2*k1[:]
    y_derivs = dydx(x_c2, y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 valuea
    y_temp[:] = yi[:] + 0.075*k1[:]+ 0.225*k2[:]
    y_derivs = dydx(x_c3, y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 valuea
    y_temp[:] = yi[:] + 0.3*k1[:] - 0.9*k2[:] + 1.2*k3[:]
    y_derivs = dydx(x_c4, y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] -11/54.*k1[:] + 2.5*k2[:] -70/27.*k3[:] +35/27.*k4[:]
    y_derivs = dydx(x_c5, y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] +1631/55296.*k1[:] + 175/512.*k2[:] +575/13824.*k3[:] +44275/110592.*k4[:] + 253/4096.*k5[:]
    y_derivs = dydx(x_c6, y_temp)
    k6[:] = h*y_derivs[:]
    
    
    #advance y by a step h 
    yipo = yi + 37/378.*k1 + 0*k2 + 250/621.*k3 + 125/594.*k4 + 0 * k5 + 512/1771.*k6
    
    yipo2 = yi + 2825/27648*k1 + 0*k2 + 18575/48384.*k3 + 13525/55296.*k4 + 277/14336 * k5 + 1/4.*k6
    
    return yipo, yipo2
    
    

### adaptive step set driver ! :D

In [ ]:
def rk5_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #max number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0 
    
    #what the current error estimate is for each variable
    #set a bigger than tolerance to start 
    Delta = np.full(nv, 2*tol)
    
    
    #remember the current step size
    h_step = h
    
    
    
    
    #adjust step if neccessary 
    while(Delta.max()/tol > 1.0):
        
        #using special adaptive step size control to estimate error
        y_step, ystar_step = rk5_mv_core(dydx, x_i, y_i, nv, h_step)
        Delta = np.fabs(y_step - ystar_step)
        
        #if error is too large, take a smaller step
        
        if(Delta.max()/tol>1.0):
            
            #error too large
            #reduce step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iterations
        if(i>=imax):
            
            print("too many iterations in rk_mv_ad()")
            raise StopIteration("Ending after i =", i)
            
        #iterate
        i += 1
        
        
    #out of loop
    
    #next time try a bigger step
    h_new = np.fmin(h_step*(Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    
    #return the answer, a new step, and the step we took 
    return y_step, h_new, h_step
    
        

### defining a wrapper ! :D

In [ ]:
def rk5_mv(dydx, a, b, y_a, tol):
    
    #dfdx is the derivative wrt x 
    # a is the lower bound
    # b is the uppe bound
    # y_a are the boundary conditions at a 
    #tol is the error tolerance for y 
    
    xi = a
    yi = y_a.copy()
    
    #an initla step
    
    h = 1.0e-4*(b-a)
    
    imax = 100000
    
    #iteration varaibel
    i = 0
    
    #set the number of coupled odes to the size of y_a
    
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1, a)
    
    y = np.full((1, nv), y_a)
    
    #set a flag 
    flag = 1
    
    #loop until we reach b 
    while(flag):
        
        yi_new, h_new, h_step = rk5_mv_ad(dydx, xi, yi, nv, h, tol)
        
        #update the step
        
        h = h_new 
        
        #prevent an overshoot
        
        if(xi+h_step>b):
            
            h = b - xi
            
            yi_new, h_new, h_step = rk5_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0 
            
        #update value values
        xi += h_step
        yi[:] = yi_new[:]
        
        
        #add tge srteps to the array 
        x = np.append(x, xi)
        
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x)-1, :] = y
        y_new[-1, :] = yi[:]
        del y
        y = y_new
        
        #prevent too many irerations
        
        if(i>=imax):
            print("maximum itertaons reached")
            raise StopIterations("iteration number = ", i)
            
        #iterate 
        i +=1
        
        #output some info
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" % (9, xi, h_step, b)
        print(s)
        
        #break if new xi == b
        if(xi==b):
            flag = 0
    #outside of while loop      
    return x,y

### perform the intergration! :D

In [ ]:
a = 0.0
b = 2.0* np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

x,y = rk5_mv(dydx, a, b, y_0, tolerance)

### plotting the result! :D

In [ ]:
plt.plot(x, y[:,0], 'o', label='y(x)')
plt.plot(x, y[:,1], 'o', label = 'dydx(x)')
xx = np.linspace(0,2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label='sin(x)')
plt.plot(xx, np.cos(xx), label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)
         


### plotting the error! 

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)